In [2]:
import geopandas as gpd
import pandas as pd
import json
import copy
import os

### Konfiguration

In [3]:
state = 'PA'

In [4]:
pos = {
    'IA': {
        'name': 'Iowa',
        'scale': 2530,
        'translate': [80, 310],
        'arrow': 2
    },
    'PA': {
        'name': 'Pennsylvania',
        'scale': 4000,
        'translate': [-800, 380],
        'arrow': 2
    },
    'WI': {
        'name': 'Wisconsin',
        'scale': 2500,
        'translate': [-20,410],
        'arrow': 2
    },
    'MI': {
        'name': 'Michigan',
        'scale': 2070,
        'translate': [-90,370],
        'arrow': 2
    }
}

### Daten runterfiltern

In [5]:
d1 = gpd.GeoDataFrame(pd.read_pickle('temp/windofchange.pkl'))
d1

,fips,county,state,geometry,winner,shifted,population,_merge,datensatz
0,13027,Brooks,GA,"MULTIPOLYGON (((-83.73616 31.03768, -83.57396 ...",Republicans,4.0,16245.0,both,NaN
1,31095,Jefferson,NE,"MULTIPOLYGON (((-97.36869 40.35039, -96.91606 ...",NaN,2.0,7054.0,both,NaN
2,31095,Jefferson,NE,"MULTIPOLYGON (((-97.36869 40.35039, -96.91606 ...",NaN,0.0,7054.0,both,NaN
3,51683,Manassas,VA,"MULTIPOLYGON (((-77.52666 38.73320, -77.49545 ...",NaN,-22.0,42696.0,both,NaN
4,56021,Laramie,WY,"MULTIPOLYGON (((-105.27824 41.65666, -104.6533...",NaN,-24.0,100984.0,both,NaN
...,...,...,...,...,...,...,...,...,...
3338,9007,NaN,NaN,None,Republicans,4.0,NaN,right_only,Nikolai
3339,9008,NaN,NaN,None,Democrats,-9.0,NaN,right_only,Nikolai
3340,9009,NaN,NaN,None,Republicans,4.0,NaN,right_only,Nikolai
3341,9010,NaN,NaN,None,Democrats,12.0,NaN,right_only,Nikolai


In [6]:
d2 = d1[d1.state == state]
d2

,fips,county,state,geometry,winner,shifted,population,_merge,datensatz
11,42063,Indiana,PA,"MULTIPOLYGON (((-79.45018 40.53015, -79.21524 ...",NaN,-2.0,83094.0,both,NaN
47,42053,Forest,PA,"MULTIPOLYGON (((-79.51207 41.62456, -78.95614 ...",NaN,-7.0,6449.0,both,NaN
104,42011,Berks,PA,"MULTIPOLYGON (((-76.44018 40.49595, -76.17162 ...",NaN,11.0,432821.0,both,NaN
190,42093,Montour,PA,"MULTIPOLYGON (((-76.73489 41.09213, -76.73267 ...",NaN,1.0,17860.0,both,NaN
458,42065,Jefferson,PA,"MULTIPOLYGON (((-79.21072 41.03242, -79.21521 ...",NaN,-9.0,43612.0,both,NaN
...,...,...,...,...,...,...,...,...,...
3156,42119,Union,PA,"MULTIPOLYGON (((-77.36418 40.84694, -77.27924 ...",NaN,-7.0,42042.0,both,NaN
3164,42089,Monroe,PA,"MULTIPOLYGON (((-75.64941 41.12230, -75.60074 ...",NaN,4.0,166053.0,both,NaN
3214,42071,Lancaster,PA,"MULTIPOLYGON (((-76.69766 40.15634, -76.67814 ...",NaN,-15.0,558589.0,both,NaN
3284,42095,Northampton,PA,"MULTIPOLYGON (((-75.57749 40.79941, -75.47419 ...",NaN,2.0,319091.0,both,NaN


In [7]:
d2.to_file('temp/'+state+'.geojson', driver='GeoJSON')

In [8]:
!geo2topo temp/{state}.geojson>temp/{state}.topo.json

### Arrowmap

In [9]:
with open('temp/'+state+'.topo.json', 'r') as f:
    stateData = json.load(f)

In [10]:
with open('specs/arrow-mw.vg.json', 'r') as f:
    spec_mw = json.load(f)

In [11]:
spec_mw['data'][1]['values'] = stateData
spec_mw['data'][1]['format']['feature'] = state
spec_mw['data'][0]['transform'] = [
        {
          "type": "filter",
          "expr": "datum.properties.name === '"+pos[state]['name']+"'"
        }
      ]
spec_mw['projections'][0]['scale'] = pos[state]['scale']
spec_mw['projections'][0]['translate'] = pos[state]['translate']
spec_mw['signals'][0]['value'] = pos[state]['arrow']

In [12]:
folder_path = 'temp/specs/'

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

In [13]:
with open('temp/specs/arrow-'+state+'-mw.vg.json', 'w') as f:
    json.dump(spec_mw, f, indent=4)

### Bubblemap

In [14]:
with open('specs/bubble-mw.vg.json', 'r') as f:
    spec_mw = json.load(f)

In [15]:
spec_mw['data'][1]['values'] = stateData
spec_mw['data'][1]['format']['feature'] = state
spec_mw['data'][0]['transform'] = [
        {
          "type": "filter",
          "expr": "datum.properties.name === '"+pos[state]['name']+"'"
        }
      ]
spec_mw['projections'][0]['scale'] = pos[state]['scale']
spec_mw['projections'][0]['translate'] = pos[state]['translate']

In [16]:
with open('temp/specs/bubble-'+state+'-mw.vg.json', 'w') as f:
    json.dump(spec_mw, f, indent=4)

In [17]:
!vg2svg temp/specs/arrow-{state}-mw.vg.json svgs/arrow-{state}-mw.svg
!vg2svg temp/specs/bubble-{state}-mw.vg.json svgs/bubble-{state}-mw.svg

(node:57224) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.
(Use `node --trace-deprecation ...` to show where the warning was created)
(node:57226) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.
(Use `node --trace-deprecation ...` to show where the warning was created)
